# Assignment 4: Encoding models (10 pts in total)

In [1]:
# Install torchextractor for facilitating feature extraction
!pip install torchextractor

# Import packages
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter('ignore')

import torch
from torchvision import models, transforms
import torchextractor as tx
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold

In [2]:
# Download the dataset
!gdown 1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS
# Extract the dataset and remove the tar file
!mkdir -p natural_scenes_demo && tar -xzf natural_scenes_demo.tar.gz -C natural_scenes_demo
!rm natural_scenes_demo.tar.gz

Downloading...
From (original): https://drive.google.com/uc?id=1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS
From (redirected): https://drive.google.com/uc?id=1q5mPgOpEsWx4x_FeuZWu-8HfsdGr-aLS&confirm=t&uuid=d015df94-8421-4319-8bbb-9335f21150ca
To: /content/natural_scenes_demo.tar.gz
100% 395M/395M [00:10<00:00, 37.9MB/s]
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
tar: Ignoring unknown extended header keyword 'SCHILY.fflags'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.FinderInfo'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown ext

### Dataset description
This dataset contains only one subject, who viewed 1000 images in the fMRI scanner. The images are naturalistic images. The fMRI data has been preprocessed, and voxel-wise BOLD responses has been estimated. The voxel-wise BOLD responses are stored in the file 'response_data.parquet', with rows representing voxels and columns representing images. The file 'metadata.csv' records detailed information on each voxel.

The file 'stimulus_data.csv' records detailed information on each image. The image files are located at the folder 'stimulus_set'.

In this assignment, you will fit an encoding model with AlexNet activations to BOLD responses in the region PPA.

The BOLD responses in PPA have been extracted for you with the code below.

In [5]:
# Load voxel metadata
df = pd.read_csv('natural_scenes_demo/metadata.csv')

# Select voxels in PPA with reliability > 0.1
voxel_ids = df.loc[
    (df['roi_name'] == 'PPA') & (df['voxel_reliability'] > 0.1),
    'voxel_id'
].to_numpy()

# Load voxel-wise BOLD responses
data = pd.read_parquet('natural_scenes_demo/response_data.parquet')
data = data.set_index('voxel_id')
bold_responses = data.loc[voxel_ids].to_numpy()  # shape: (num_voxels, num_images)
print(f'Number of voxels in PPA with reliability > 0.1: {len(voxel_ids)}')
print(f'BOLD responses shape: {bold_responses.shape}')

Available columns: ['image_id', 'image_name', 'image_path', 'coco_id', 'coco_supercategs', 'coco_categs', 'coco_areas', 'coco_captions']
First few rows:
      image_id               image_name  \
0  coco_264244  shared0014_nsd03627.png   
1  coco_267699  shared0036_nsd04787.png   
2  coco_268114  shared0040_nsd04931.png   
3  coco_531392  shared0045_nsd05302.png   
4  coco_270278  shared0053_nsd05603.png   

                                          image_path  coco_id  \
0  /content/natural_scenes_demo/stimulus_set/shar...   264244   
1  /content/natural_scenes_demo/stimulus_set/shar...   267699   
2  /content/natural_scenes_demo/stimulus_set/shar...   268114   
3  /content/natural_scenes_demo/stimulus_set/shar...   531392   
4  /content/natural_scenes_demo/stimulus_set/shar...   270278   

                                    coco_supercategs  \
0  ['animal', 'person', 'furniture', 'electronic'...   
1  ['vehicle', 'vehicle', 'vehicle', 'vehicle', '...   
2                  ['vehicle'

#### ✏️ Do it yourself (0.5 pts):
How is reliablity of BOLD responses generally calculated? Why do we want to remove voxels with low reliability?

Write your answer here:
> **Answer:**
> The reliability of BOLD responses is calculated by collecting the same responses at different times (test-retest reliability). The correlation between responses to the same stimuli across different sessions provides a measure of reliability.
>   We want to remove voxels with low reliability because they represent the noise ceiling, which is the upper bound of explainable variance. Noise levels will be different at different times of day, so the measured response will be different.


#### ✏️ Do it yourself (2 pt):
Exact layer activations of 1000 images from the last convolutional layer (after ReLU) in Alexnet.
_Hint: Make sure the layer you want is properly indexed_

In [10]:
# Insert your code here

# Load AlexNet pretrained model (fix deprecation warning)
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
alexnet.eval()

# Let's first check the AlexNet architecture to find the correct layer
print("AlexNet features layers:")
for i, layer in enumerate(alexnet.features):
    print(f"features.{i}: {layer}")

# The last convolutional layer in AlexNet is features.10 (Conv2d(256, 256, kernel_size=(3, 3)))
# followed by features.11 (ReLU)
# Extract features from features.10 and we'll apply ReLU ourselves
model = tx.Extractor(alexnet, ["features.10"])

# Define image preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load stimulus information
stimulus_df = pd.read_csv('natural_scenes_demo/stimulus_data.csv')

# Extract activations for all 1000 images
activations_list = []
for idx in tqdm(range(len(stimulus_df)), desc="Extracting features"):
    img_filename = stimulus_df.iloc[idx]['image_name']
    img_path = f"natural_scenes_demo/stimulus_set/{img_filename}"
    img = Image.open(img_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)

    with torch.no_grad():
        _, features = model(img_tensor)
        # Apply ReLU activation (features.11 in the original architecture)
        activation = torch.relu(features['features.10'])
        activations_list.append(activation.squeeze().cpu().numpy())

# Stack all activations
activations_conv5 = np.stack(activations_list)
print(f'\nActivations shape: {activations_conv5.shape}')
print(f'Expected shape: (1000, 256, 13, 13) = 256 channels × 13 × 13 spatial dimensions')

# flatten the activations into vectors
activations_conv5_flat = activations_conv5.reshape(len(activations_conv5), -1)
print(f'Flattened activations shape: {activations_conv5_flat.shape}')
print(f'Expected flattened shape: (1000, 43264) where 43264 = 256 × 13 × 13')

AlexNet features layers:
features.0: Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
features.1: ReLU(inplace=True)
features.2: MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
features.3: Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
features.4: ReLU(inplace=True)
features.5: MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
features.6: Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
features.7: ReLU(inplace=True)
features.8: Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
features.9: ReLU(inplace=True)
features.10: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
features.11: ReLU(inplace=True)
features.12: MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)


Extracting features: 100%|██████████| 1000/1000 [00:43<00:00, 22.85it/s]


Activations shape: (1000, 256, 13, 13)
Expected shape: (1000, 256, 13, 13) = 256 channels × 13 × 13 spatial dimensions
Flattened activations shape: (1000, 43264)
Expected flattened shape: (1000, 43264) where 43264 = 256 × 13 × 13


#### ✏️ Do it yourself (0.5 pt):
You should have extracted 43264 features from each image in the data set from conv5. Now randomly split 80% of the data as the training set and 20% as the testing set. \
_Hint: Use `train_test_split` from sklearn. Set random_state=42 so the split are reproducible_

In [11]:
# Insert your code here

X_train, X_test, y_train, y_test = train_test_split(
    activations_conv5_flat,  # Features (CNN activations)
    bold_responses.T,        # Target (BOLD responses, transposed to shape: num_images x num_voxels)
    test_size=0.2,           # 20% for testing
    random_state=42          # For reproducibility
)

print(f'Training set: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}')
print(f'Testing set: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}')

Training set: X_train shape = (800, 43264), y_train shape = (800, 399)
Testing set: X_test shape = (200, 43264), y_test shape = (200, 399)


#### ✏️ Do it yourself (1 pt):
Now you want to do dimension reduction by performing PCA on the 43264 features and reduce it to 50 dimensions. \

Why do you want to do dimension reduction? Should you perform PCA only on the training dataset or on the whole dataset, and why?

Write your answer here:
> **Answer:**
> We want to do dimension reduction to mitigate the curse of dimensionality. When samples << dimensionality (800 training vs 43k features), we're prone to overfitting to noise (i.e. irrelevant features) and embed that noise into the model. Thus, to improve stability and generalizability, we need to do dimension reduction first.
>   You should do PCA only on training set because you need to save maybe 10-20% of the entire dataset for testing the model after training.

#### ✏️ Do it yourself (1 pt):
Perform PCA to reduce the dimension to 50.

In [12]:
# Insert your code here
# Insert your code here

# Initialize PCA with 50 components
pca = PCA(n_components=50)

# Fit PCA ONLY on training data
pca.fit(X_train)

# Transform both training and test data using the fitted PCA
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

print(f'Training set after PCA: {X_train_pca.shape}')
print(f'Testing set after PCA: {X_test_pca.shape}')
print(f'Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}')


Training set after PCA: (800, 50)
Testing set after PCA: (200, 50)
Explained variance ratio: 0.2633


#### ✏️ Do it yourself (1 pt):
Build encoding models by conducting voxel-wise Ridge regression and predicting on the test dataset. \
_Hint: Use `Ridge` from sklearn. Set the alpha = 1_

In [13]:
# Insert your code here

# Initialize Ridge regression with alpha=1
ridge = Ridge(alpha=1)

# Fit the model on training data
ridge.fit(X_train_pca, y_train)

# Predict on test data
y_pred = ridge.predict(X_test_pca)

print(f'Predictions shape: {y_pred.shape}')
print(f'Model coefficients shape: {ridge.coef_.shape}')

# Optional: Calculate performance metrics
from sklearn.metrics import r2_score
import numpy as np

# Calculate R² for each voxel
r2_scores = []
for voxel_idx in range(y_test.shape[1]):
    r2 = r2_score(y_test[:, voxel_idx], y_pred[:, voxel_idx])
    r2_scores.append(r2)

r2_scores = np.array(r2_scores)
print(f'\nMean R² across voxels: {r2_scores.mean():.4f}')
print(f'Median R² across voxels: {np.median(r2_scores):.4f}')
print(f'Max R² across voxels: {r2_scores.max():.4f}')

Predictions shape: (200, 399)
Model coefficients shape: (399, 50)

Mean R² across voxels: 0.2195
Median R² across voxels: 0.2086
Max R² across voxels: 0.5130


#### ✏️ Do it yourself (1 pt):
Evaluate the performance of your encoding model by computing the correlation coefficient. Get a summary score for the ROI (i.e., PPA) by calculating the _median_ of all voxels. \
_Hint: Use np.corrcoef to compute correlation_

In [14]:
# Insert your code below

# compute r for each voxel
r_scores = []
for voxel_idx in range(y_test.shape[1]):
    # np.corrcoef returns a 2x2 correlation matrix
    # [0, 1] or [1, 0] gives the correlation between the two variables
    corr_matrix = np.corrcoef(y_test[:, voxel_idx], y_pred[:, voxel_idx])
    r = corr_matrix[0, 1]
    r_scores.append(r)

r_scores = np.array(r_scores)

# compute the median r across voxels
median_r = np.median(r_scores)

print(f'Correlation scores (r) for each voxel:')
print(f'Mean r: {r_scores.mean():.4f}')
print(f'Median r: {median_r:.4f}')
print(f'Max r: {r_scores.max():.4f}')
print(f'Min r: {r_scores.min():.4f}')

Correlation scores (r) for each voxel:
Mean r: 0.4653
Median r: 0.4622
Max r: 0.7251
Min r: 0.1133


#### ✏️ Do it yourself (3 pt):
Now you will wrap up all those steps above into one code cell, and put the encoding models under a _two-folds_ cross-validation scheme. Note that to get the final performance evaluation, concatenate each fold into one big data and compute the correlation.Output the median r of all voxels.\
_Hint: Use `KFold` to split folds. Remember to set random_state=42 for reproducibility_

In [15]:
# Insert your code here
# Insert your code below

# Initialize KFold with 2 folds
kfold = KFold(n_splits=2, shuffle=True, random_state=42)

# Lists to store predictions and true values from each fold
all_y_true = []
all_y_pred = []

print("Running 2-fold cross-validation...")

# Iterate through the folds
for fold_idx, (train_idx, test_idx) in enumerate(kfold.split(activations_conv5_flat)):
    print(f"\nFold {fold_idx + 1}/2")

    # Split data into train and test for this fold
    X_train_fold = activations_conv5_flat[train_idx]
    X_test_fold = activations_conv5_flat[test_idx]
    y_train_fold = bold_responses.T[train_idx]  # Transpose to (num_images, num_voxels)
    y_test_fold = bold_responses.T[test_idx]

    print(f"  Train size: {X_train_fold.shape[0]}, Test size: {X_test_fold.shape[0]}")

    # Step 1: PCA dimension reduction (fit on training data only)
    pca = PCA(n_components=50)
    pca.fit(X_train_fold)
    X_train_pca = pca.transform(X_train_fold)
    X_test_pca = pca.transform(X_test_fold)

    print(f"  PCA explained variance: {pca.explained_variance_ratio_.sum():.4f}")

    # Step 2: Ridge regression (fit on training data)
    ridge = Ridge(alpha=1)
    ridge.fit(X_train_pca, y_train_fold)

    # Step 3: Predict on test data
    y_pred_fold = ridge.predict(X_test_pca)

    # Store predictions and true values for this fold
    all_y_true.append(y_test_fold)
    all_y_pred.append(y_pred_fold)

# Concatenate predictions and true values from all folds
all_y_true = np.vstack(all_y_true)
all_y_pred = np.vstack(all_y_pred)

print(f"\nConcatenated results shape:")
print(f"  True values: {all_y_true.shape}")
print(f"  Predictions: {all_y_pred.shape}")

# Compute correlation for each voxel across all concatenated data
r_scores_cv = []
for voxel_idx in range(all_y_true.shape[1]):
    corr_matrix = np.corrcoef(all_y_true[:, voxel_idx], all_y_pred[:, voxel_idx])
    r = corr_matrix[0, 1]
    r_scores_cv.append(r)

r_scores_cv = np.array(r_scores_cv)

# Compute the median r across voxels
median_r_cv = np.median(r_scores_cv)

print(f"\n{'='*50}")
print(f"2-Fold Cross-Validation Results:")
print(f"{'='*50}")
print(f"Mean r across voxels: {r_scores_cv.mean():.4f}")
print(f"Median r across voxels: {median_r_cv:.4f}")
print(f"Max r across voxels: {r_scores_cv.max():.4f}")
print(f"Min r across voxels: {r_scores_cv.min():.4f}")
print(f"Std r across voxels: {r_scores_cv.std():.4f}")


Running 2-fold cross-validation...

Fold 1/2
  Train size: 500, Test size: 500
  PCA explained variance: 0.3055

Fold 2/2
  Train size: 500, Test size: 500
  PCA explained variance: 0.3034

Concatenated results shape:
  True values: (1000, 399)
  Predictions: (1000, 399)

2-Fold Cross-Validation Results:
Mean r across voxels: 0.4425
Median r across voxels: 0.4372
Max r across voxels: 0.6644
Min r across voxels: 0.1330
Std r across voxels: 0.0962
